<img src="https://github.com/jupytercon/2020-exactlyallan/raw/master/images/RAPIDS-header-graphic.png" style="width:50%">


# RAPIDS Visualization Guide Notebook
### A Streamlined Guide to RAPIDS Accelerated Visualization and Visual Analtyics
The guide will walk through using RAPIDS cuDF, cuSpatial, and cuGraph with Holoviews, hvPlot, Datashader, cuxfilter, and Plotly Dash with the publically availble Divvy Bike share dataset. 

**NOTES and TODO:**
-Base on [JupyterCon Notebooks](https://github.com/rapidsai-community/event-notebooks/blob/main/JupyterCon_2020_RAPIDSViz/00%20Index%20and%20Introduction.ipynb) not cuxfilter tutorial


## Requirements
- System that meets the [RAPIDS system and GPU requirements](https://docs.rapids.ai/install#system-req)


## Dependencies
Use the below to install all the required dependencies via conda:

channels:
- rapidsai
- nvidia
- pyviz
- conda-forge
- plotly
- anaconda

dependencies:
- cuxfilter>=23.02
- cudf>=23.02
- cuspatial>=23.02
- cugraph>=23.02
- cudatoolkit=11.8
- python>=3.10
- plotly
- dash-core-components
- dash-html-components
- jupyter-dash
- jupyterlab
- jupyter-server-proxy
- holoviews
- hvplot
- geoviews
- cartopy
- networkx

In [ ]:
# imports
import os
from zipfile import ZipFile
from pathlib import Path

import cudf


## Dataset
The dataset can be downloaded from the [Divvy Bike Share public dataset](https://divvybikes.com/system-data). Use the following script to download the desired date range and load it into a dataframe.


In [ ]:
# Define the URL of the Divvy trip data and save dir
S3 = 'https://divvy-tripdata.s3.amazonaws.com/'
DATA_DIR = './data'


In [ ]:
# Check dir
Path(DATA_DIR).mkdir(parents=True, exist_ok=True)

# Download the zip files from the URL within date range and unzip
for year in range(2021, 2022):
    for month in range(1, 3):
        file = f'{year}{month:02d}-divvy-tripdata.zip'
        URL = f'{S3}{file}'
        ! wget -P {DATA_DIR} {URL}
     
        with zipfile.ZipFile(f'{DATA_DIR}/{file}') as zip:
            zip.extractall(f'{DATA_DIR}')
            

In [ ]:
# Load all csv as dataframes and combine into one cudf
df_array = []

for file in Path(DATA_DIR).rglob('*.csv'):
    gdf = cudf.read_csv(file)
    df_array.append(gdf)

df = cudf.concat(df_array)

# Check the data
df

The data seems unreasonabliy clean, but there are still a few things we improve on it. First lets double check the dtypes.


In [ ]:
df.dtypes

The 'started_at' and 'ended_at' columns should be proper date times types.

In [ ]:
df['started_at'] = cudf.to_datetime(df['started_at'])
df['ended_at'] = cudf.to_datetime(df['started_at'])

df.dtypes

To make things a bit easier lets break out the date and time into sperate columns, assuming we only need to worry about start time.

In [ ]:
df['year'] = df['started_at'].dt.year
df['month'] = df['started_at'].dt.month
df['day'] = df['started_at'].dt.day
df['hour'] = df['started_at'].dt.hour

df

Finding the duration of each trip would also be a helpful metric.

In [ ]:
## only listing zeros... 
df['duration'] = df['ended_at'] - df['started_at']

df

Extracting out the day of the week would be hepful too.

In [ ]:
df['day_of_week'] = df['started_at'].dt.dayofweek

df

Lets map that to more legible names

In [ ]:
dow = {0:'M', 1:'T', 2:'W', 3:'Th', 4:'F', 5:'Sa', 6:'Su'}
df["day_of_week"] = df["day_of_week"].map(dow)

df

In [ ]:

df.groupby("member_casual").size()
